In [1]:
import pandas.io.sql as psql
import pymysql


In [34]:
COMMENT HERE TO RUN, THIS CAN TAKE COUPLE OF HOURS 

connection = pymysql.connect(host='analytics-store.eqiad.wmnet', 
                             db='wikidatawiki',
                             read_default_file='/etc/mysql/conf.d/research-client.cnf',
                             cursorclass=pymysql.cursors.DictCursor)

sql = "SELECT ips_item_id,ips_site_id,ips_site_page FROM wb_items_per_site"
df = psql.read_sql_query(sql, connection)
df['ips_site_id'] = df.ips_site_id.str.decode('utf-8')
df['ips_site_page'] = df.ips_site_page.str.decode('utf-8')
df.to_pickle('wikidataItems.p')
df.to_csv('wikidataItems.csv')

In [35]:
!hadoop fs -put wikidataItems.csv

In [2]:
from pyspark.sql.types import StructType, StructField, LongType
from pyspark.sql.types import DoubleType, IntegerType, StringType

schema = StructType([
    StructField("index", IntegerType()),
    StructField("Qitem", LongType()),
    StructField("wiki", StringType()),
    StructField("page", StringType())
])

In [3]:
pdf = spark.read.csv('wikidataItems.csv',schema=schema,header=True).select('Qitem','wiki','page')

In [ ]:
pdf.write.saveAsTable('diego.WikiData')

In [26]:
pdf.show()

+-------+-------+--------------------+
|  Qitem|   wiki|                page|
+-------+-------+--------------------+
|3596065| abwiki|            Џьгьарда|
|3596033| abwiki|                Аацы|
| 620894| abwiki|          Агәы-Бедиа|
| 621901| abwiki|               Ахәри|
| 621898| abwiki|             Аҽгәара|
| 621890| abwiki|             Баслахә|
|3596028| abwiki|           Блабырхәа|
| 620880| abwiki|             Галхәыч|
| 621896| abwiki|               Гәдаа|
|3596049| abwiki|Казан аметрополит...|
| 621889| abwiki|              Кындыҕ|
| 620887| abwiki|              Махәыр|
| 620878| abwiki|    Тҟәарчал (ақыҭа)|
|3596038| abwiki|               Уатаҧ|
| 620884| abwiki|            Чхәарҭал|
|4151202| abwiki|               Ҕәада|
| 620882| abwiki|            Ҕәымрышь|
| 620890| abwiki|               Ҵарча|
|3596018| abwiki|                Ҷлоу|
|  47467|acewiki|     Gunong Perkison|
+-------+-------+--------------------+
only showing top 20 rows



In [32]:
pdfFilter = pdf.where(pdf.wiki.isin(['arwiki','enwiki','ruwiki','eswiki','jawiki','frwiki']))

In [33]:
pdfFilter.show()

+-------+------+--------------------+
|  Qitem|  wiki|                page|
+-------+------+--------------------+
|4545892|arwiki|10000 شجرة للوادي...|
|4632674|arwiki|               28-بت|
| 333962|arwiki|                   آ|
|2876523|arwiki|            آث وغليس|
|4120101|arwiki|                  آج|
|2390046|arwiki|        آرثور ج. ماي|
|4793689|arwiki|               آرمور|
|4120414|arwiki|      آشور بيت سركيس|
|2401317|arwiki|                آشير|
|2615540|arwiki|            آل نهيان|
| 356868|arwiki|           آلة كظومة|
|2850512|arwiki|           آنا غريكا|
|4118564|arwiki|        آي آر سي إكس|
|1669566|arwiki|           آية الرجم|
|2657293|arwiki|   آيت توزين (قبيلة)|
|2876451|arwiki|            آيت عياش|
|3147588|arwiki|           آيس بي بي|
|4119053|arwiki|        أبان بن تغلب|
|2839581|arwiki|أبجدية صوتية دولي...|
|2839568|arwiki| أبجدية وطنية تشادية|
+-------+------+--------------------+
only showing top 20 rows



In [34]:
pdfFilter.write.saveAsTable('diego.WikiDataSixLangs')

In [1]:
#Retun wikidata Six Langs to File
sixLangs = sqlContext.sql('SELECT CONCAT("Q",Qitem), replace(wiki,"wiki",""), page FROM diego.WikiDataSixLangs')

In [4]:
sixLangs.write.csv('wikidataSixLanguages.csv.gz',sep='\t',compression='gzip')

In [6]:
sixLangs.count()

16600427